In [ ]:

class Trainer:

    def __init__(self, config, model, train_dataset):
        self.config = config
        self.model = model
        self.optimizer = None
        self.train_dataset = train_dataset

        self.device = 'cpu'
        self.model = self.model.to(self.device)
        self.iter_num = 0
        self.iter_time = 0.0
        self.iter_dt = 0.0

    def add_callback(self, onevent: str, callback):
        self.callbacks[onevent].append(callback)

    def set_callback(self, onevent: str, callback):
        self.callbacks[onevent] = [callback]

    def trigger_callbacks(self, onevent: str):
        for callback in self.callbacks.get(onevent, []):
            callback(self)

    def run(self):
        model, config = self.model, self.config
        device = "cpu"
        self.optimizer = model.configure_optimizers(config)

        def get_batch():
          data = self.train_dataset
          block_size = 100
          ix = torch.randint(data.shape[0]-block_size,(config.batch_size,))
          x = torch.stack([data[i:i+block_size] for i in ix])
          y = torch.stack([data[i+1:i+block_size+1] for i in ix])
          x,y = x.to(device),y.to(device)
          return x,y

        @torch.no_grad
        def cross_val():
          model.eval()
          losses = torch.zeros(200+1)
          for k in range(200):
            X,Y = get_batch()
            logits,loss = model(X,Y)
            losses[k]=loss.item()
            out = losses.mean()
          model.train()
          return out

        model.train()
        for epoch in range(15):
          print("[epoch {o}] \n".format(o = epoch))

          for i in range(50):
            x, y = get_batch()
            logits, self.loss = model(x, y)

            model.zero_grad(set_to_none=True)
            self.loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
            self.optimizer.step()
          print("\n[train loss = {k} ]\n".format(epoch = epoch , k = cross_val()))

